# Purpose

This notebook will find the 50k most active Lichess players (rapid blitz classical), measured by number of games.

It does this by downloading and reading parquet files of raw game data.

It's not practical to download the hundreds of 1GB parquet files; so we will download a certain number and extrapolate from there.

We don't need to know exactly who is the *most* active, just need to find *very* active players.

Then, we will process their games for data to feed in to our chess opening recommender AI model.

# Reason

Originally, we were just collecting data on millions of players without caring who was active or not.

This grew impractical quickly, with a massive local DB. Games processing slowed down exponentially because SQL queries to the local duckdb got so slow due to the db's size.

So I've decided to focus on the 50k most active players instead.

# Method

1. Get names of parquet files
    - We will want to look at games data over the course of a year or so
    - So, we'll download some 1GB raw game data parquet files (number TBD) from each month (there are about 60-70 total per month)
    - To do this, we need the names of those files to get from the HuggingFace API
    - Luckily we already have functionality elsewhere to do this; I'll import it here and update slightly for our needs

2. Download parquet files
    - Once we have the names of the needed files, we'll download them
    - I already have functionality to do this elsewhere, just need to import it here

3. Save game counts
    - Once we have our parquet files downloaded, we'll count the number of games each username has
    - There will be some filters; only Rapid/Blitz/Classical games, only players over 1200 rating
    - Need to decide on a data saving method; probably in a CSV with username/num_games
    - Wherever it's saved it needs to be persistent, as this processing will happen over multiple sessions.

4. Get most active players
    - Finally, we'll analyze our CSV to retrieve the usernames of the 50,000 most active Lichess players
    - Profit


- Note that all data for this will be saved in data/processed/find_most_active_players directory.

## Config

Below, we'll define some important variables that we want to use in this notebook.

I like to keep them in one place at the top.

In [ ]:
from utils.downloading_raw_parquet_data.get_parquet_file_names import (
    get_parquet_file_names,
)

# The year and month we want to download data for

month_for_downloading = 6
year_for_downloading = 2025

max_files_to_download_per_month = 3

## 1. Get file names

Now, we'll get the file names for the month and year we want to download.

In [ ]:
file_names_for_month = get_parquet_file_names(year_for_downloading, month_for_downloading)
# print with line separations
print("\n".join(file_names_for_month[:max_files_to_download_per_month]))

## Database Setup

Now, we'll initialize our database.

This is a small database that tracks the number of games played by a Lichess username, as well as which files have been checked so there are no duplicates.

In [ ]:
import os
from pathlib import Path
from utils.database.db_utils import get_db_connection
from utils.database.player_game_counts_db_utils import setup_player_game_counts_table

# Define the project root
project_root = Path.cwd().parent

# Initialize the database
DB_PATH = project_root / "data" / "processed" / "find_most_active_players" / "player_game_counts.duckdb"
DB_DIR = DB_PATH.parent

# Ensure the directory exists
if not DB_DIR.exists():
    DB_DIR.mkdir(parents=True)
    print(f"Created directory: {DB_DIR}")

# Ensure the database file exists
if not DB_PATH.exists():
    print(f"Database file {DB_PATH} does not exist. Initializing...")
    con = get_db_connection(str(DB_PATH))
    setup_player_game_counts_table(con)
    con.close()
    print("Database created and initialized.")
else:
    print(f"Database file {DB_PATH} already exists. Skipping initialization.")

## Main Pipeline
Now that we have the names of the files we need, we will do the following for each file name (until we've downloaded our predefined max number of files for the given month)

1. Check our local db to make sure that we haven't already downloaded the file in question
    - If we have, cycle through that month's list of files until we find one that hasn't been downloaded
2. Download the file in question
    - Note that HuggingFace's API is smart enough to not re-download files we already have on our local machine, which saves me a lot of headache when doing this repeatedly for testing.
3. Mark the file as having been processed in our local db to avoid duplicates
4. Process the file, recording each player's num_games in the local db
5. Delete the file
6. Move on to next file unless we've reached our maximum number of files

Goals:

- Thorough logging, especially for how long each step takes, including games per second while processing files
- Each file is about 1.4 million rows so that helps when providing a file ETA
- Also log which file number we're on out of the max total files to download

In [ ]:
from utils.file_processing.types_and_classes import ProcessingConfig
import pandas as pd
from typing import Set
import re

# --- Processing Configuration ---
# Base config for processing. This will be used for each file.
base_config = ProcessingConfig(
    parquet_path="",  # This will be set per-file
    db_path=DB_PATH,
    min_player_rating=1200,
    max_elo_difference_between_players=100,
    allowed_time_controls={"Blitz", "Rapid", "Classical"},
)

LOG_FREQUENCY = 100_000
TOTAL_ROWS_IN_FILE = 1_400_000

def is_valid_game(row: pd.Series, config: ProcessingConfig) -> bool:
    """
    Checks if a game is valid based on the provided processing configuration.
    """
    # Check for BOTs
    if (pd.notna(row["WhiteTitle"]) and "BOT" in row["WhiteTitle"]) or \
       (pd.notna(row["BlackTitle"]) and "BOT" in row["BlackTitle"]):
        return False

    # Check ratings
    if row["WhiteElo"] < config.min_player_rating or row["BlackElo"] < config.min_player_rating:
        return False

    # Check ELO difference
    if abs(row["WhiteElo"] - row["BlackElo"]) > config.max_elo_difference_between_players:
        return False

    # Check time control
    event_lower = str(row["Event"]).lower()
    if not any(re.search(r'\b' + re.escape(tc.lower()) + r'\b', event_lower) for tc in config.allowed_time_controls):
        return False
        
    # Check result
    if row["Result"] not in {"1-0", "0-1", "1/2-1/2"}:
        return False
        
    # Check for player names
    if not row['White'] or not row['Black']:
        return False

    return True

In [ ]:
import time
from utils.database.player_game_counts_db_utils import is_file_already_downloaded
from utils.downloading_raw_parquet_data.raw_parquet_data_file_downloader import (
    download_single_parquet_file,
)

processed_file_count = 0

for file_name in file_names_for_month[:1]:
    if processed_file_count >= max_files_to_download_per_month:
        print("Reached the maximum number of files to process. Stopping.")
        break

    start_time = time.time()

    # 1. Check in the local db whether the file has already been processed - if so, move on to next file
    con = get_db_connection(str(DB_PATH))
    is_already_downloaded = is_file_already_downloaded(con, file_name, year_for_downloading, month_for_downloading)
    con.close()

    elapsed_time = time.time() - start_time
    print(f"Checked file {file_name} in {elapsed_time:.2f} seconds.")

    if is_already_downloaded:
        print(f"File {file_name} has already been processed. Skipping.")
        continue

    print(f"File {file_name} has not been processed yet. Processing now.")

    # 2. If it hasn't been processed, download the file
    start_time = time.time()
    downloaded_file_path = download_single_parquet_file(
        repo_id="Lichess/standard-chess-games",
        repo_type="dataset",
        file_to_download=file_name,
        local_dir=Path("data/raw"),
        year=year_for_downloading,
        month=month_for_downloading,
    )
    elapsed_time = time.time() - start_time
    if downloaded_file_path:
        print(f"Downloaded file {file_name} in {elapsed_time:.2f} seconds.")
        processed_file_count += 1
    else:
        print(f"Failed to download file {file_name}. Skipping.")
        continue

    # 3. Mark the file as processed in the local db. Doing this part first because downloading is cheap and quick, we can just move on to the next file if something goes wrong
    # TODO this would be inconvenient during testing; reinstate once we know the rest of the code works
    # start_time = time.time()
    # con = get_db_connection(str(DB_PATH))
    # from utils.database.player_game_counts_db_utils import mark_file_as_downloaded
    # mark_file_as_downloaded(con, file_name, year_for_downloading, month_for_downloading)
    # con.close()
    # elapsed_time = time.time() - start_time
    # print(f"Marked file {file_name} as downloaded in {elapsed_time:.2f

    # 4. Process the file, updating valid player game counts in the local db
    start_processing_time = time.time()
    
    import duckdb
    from collections import Counter

    # Create a specific config for this file
    file_config = base_config.replace(parquet_path=str(downloaded_file_path))

    print(f"Processing {downloaded_file_path.name} with SQL...")

    # SQL statement to get valid games
    time_control_pattern = '|'.join([tc.lower() for tc in file_config.allowed_time_controls])
    
    with duckdb.connect() as con:
        # Get total number of games in the file to calculate percentage
        total_games_query = f"SELECT COUNT(*) FROM '{file_config.parquet_path}' WHERE ECO IS NOT NULL;"
        total_games = con.execute(total_games_query).fetchone()[0]

        query = f"""
            SELECT White, Black
            FROM '{file_config.parquet_path}'
            WHERE
                (WhiteTitle IS NULL OR WhiteTitle NOT LIKE '%%BOT%%')
                AND (BlackTitle IS NULL OR BlackTitle NOT LIKE '%%BOT%%')
                AND WhiteElo >= {file_config.min_player_rating}
                AND BlackElo >= {file_config.min_player_rating}
                AND abs(WhiteElo - BlackElo) <= {file_config.max_elo_difference_between_players}
                AND regexp_matches(lower(Event), '{time_control_pattern}')
                AND Result IN ('1-0','0-1','1/2-1/2')
                AND White IS NOT NULL AND Black IS NOT NULL AND ECO IS NOT NULL;
        """
        
        print("Executing SQL query to get valid games...")
        sql_start_time = time.time()
        valid_games_df = con.execute(query).df()
        sql_elapsed_time = time.time() - sql_start_time
        
        num_valid_games = len(valid_games_df)
        games_per_second = num_valid_games / sql_elapsed_time if sql_elapsed_time > 0 else 0
        
        print(f"SQL query finished in {sql_elapsed_time:.2f} seconds.")
        print(f"Found {num_valid_games} valid games out of {total_games} total games.")
        if total_games > 0:
            percentage_valid = (num_valid_games / total_games) * 100
            print(f"Percentage of valid games: {percentage_valid:.2f}%")
        print(f"Processing rate: {games_per_second:.0f} games/sec.")

        # Count games per player
        print("\nCounting games per player...")
        count_start_time = time.time()

        # We can query the pandas DataFrame directly with DuckDB
        count_query = """
            WITH all_players AS (
                SELECT White AS player FROM valid_games_df
                UNION ALL
                SELECT Black AS player FROM valid_games_df
            )
            SELECT player, COUNT(*) as num_games
            FROM all_players
            GROUP BY player
            ORDER BY num_games DESC;
        """
        
        player_counts_df = con.execute(count_query).df()
        
        count_elapsed_time = time.time() - count_start_time
        
        num_players = len(player_counts_df)
        players_per_second = num_players / count_elapsed_time if count_elapsed_time > 0 else 0
        
        print(f"Counted games for {num_players} unique players in {count_elapsed_time:.2f} seconds.")
        print(f"Processing rate: {players_per_second:.0f} players/sec.")
        
        print("\nTop 100 most active players in this file:")
        print(player_counts_df.head(500).to_string())

        # Now update the database with these counts
        

    # 5. Delete the file

    # 6. Move on to the next file

Checked file data/year=2025/month=06/train-00000-of-00065.parquet in 0.00 seconds.
File data/year=2025/month=06/train-00000-of-00065.parquet has not been processed yet. Processing now.
File saved to data/raw/2025-06-train-00000-of-00065.parquet
Downloaded file data/year=2025/month=06/train-00000-of-00065.parquet in 0.52 seconds.
Processing 2025-06-train-00000-of-00065.parquet with SQL...
Executing SQL query to get valid games...
File saved to data/raw/2025-06-train-00000-of-00065.parquet
Downloaded file data/year=2025/month=06/train-00000-of-00065.parquet in 0.52 seconds.
Processing 2025-06-train-00000-of-00065.parquet with SQL...
Executing SQL query to get valid games...
SQL query finished in 0.53 seconds.
Found 619561 valid games out of 1400020 total games.
Percentage of valid games: 44.25%
Processing rate: 1168643 games/sec.

Counting games per player...
Counted games for 252052 unique players in 0.09 seconds.
Processing rate: 2938553 players/sec.

Top 100 most active players in thi